# Test Generator (Girth)

Read in the candidates and item data and generate a randomised test from them using the
[Girth](https://eribean.github.io/girth/) package

We assume that the 1PL model is used.

$$
Pr(X=1) = \frac{exp(\theta-b)}{1 + exp(\theta-b)}
$$

The benefit of using the Girth library is that is it in addition to generating synthetic test data it can also be used to estimate the IRT parameters.

## Data Ingest

There are two files in the `data` folder that we need: `items.csv` and `candidates.csv`. If you want to generate a randomised set of candidates, then run the `generateCandidates` notebook first. Note: this will overwrite the `candidates.csv` file.

In [1]:
import numpy as np
from numpy.random import seed
from typing import List, Tuple
from csv import reader
import pandas as pd
from girth.synthetic import create_synthetic_irt_dichotomous
from girth import onepl_mml


def getDataAsList(datafile: str) -> List[Tuple]:
    """Turn a CSV datafile into a list of tuples

    :param datafile: the CSV file to load data from
    :return: a list of rows (tuples)
    """
    with open(datafile, 'r', encoding='utf-8-sig') as fs:
        csv_reader = reader(fs)
        row_list = list(map(tuple, csv_reader))
        return row_list[1:]    # ignore the header row
    

# convert the raw data into a simple duple of ( systemname, givenName, familyName, theta )
def getCandidates() -> List[Tuple]:
    candidates = getDataAsList('data/candidates.csv')
    new_list = [(c[0], c[1], c[2], float(c[3])) for c in candidates]
    return new_list
    

# convert the raw data into a simple triple of ( uiid, a, b )
def getItems() -> List[Tuple]:
    items = getDataAsList('data/items.csv')
    new_list = [(i[0], float(i[1]), float(i[2])) for i in items]
    return new_list

In [2]:
items = getItems()
candidates = getCandidates()

## Test Generation
We use the `create_synthetic_irt_dichotomous()` function from Girth to create the random test data. The `generateTest()` function reads data into the the numpy arrays that Girth requires, and then generates a randomised test before converting it into a pandas data frame for display.

In [3]:
def convertTupleListToArray(tl: List[Tuple], arity: int):
    l = [i[arity] for i in tl]
    return np.array(l)


def generateTest(itemList: List[Tuple], candidateList: List[Tuple], seed: int = None):
    discrimination = convertTupleListToArray(itemList, 1)
    difficulty = convertTupleListToArray(itemList, 2)
    theta = convertTupleListToArray(candidateList, 3)
    t = create_synthetic_irt_dichotomous(difficulty, discrimination, theta, seed=seed)
    return t


def convertTestToDataframe(test, itemList: List[Tuple], candidateList: List[Tuple]):
    header = []
    for i in items:
        header.append(i[0])
    rownames = []
    for i in candidates:
        rownames.append(i[0])
    df = pd.DataFrame(test.T, index=rownames, columns=header)
    return df



In [4]:
synthetic_test = generateTest(items, candidates)
testDf = convertTestToDataframe(synthetic_test, items, candidates)

In [6]:
(testDf)

,A1L_7616_01#6789,A1L_20679_02#6790,A1L_5480_03#6791,A2L_5483_04#6792,A2L_24442_05#6793,A2L_7620_06#6794,A2L_7627_07#6795,B1L_20849_08#6796,B1L_4287_09#6797,B1L_21135_10#6798,...,B1VG_3919_SA06,B2VG_5843_SA07,C1VG_4378_SA08,C2VG_8173_SA09,C2VG_18944_SA10,B2R_4464_WA_01,B2R_4161_WA_02,C1R_4135_WA_03,C1R_4421_WA_04,C1R_4136_WA_05
DT0001,1,1,1,0,0,0,1,0,1,0,...,1,1,0,0,0,0,0,0,0,0
DT0002,1,1,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
DT0003,1,1,0,1,1,1,1,1,1,1,...,1,0,1,1,0,0,1,1,0,1
DT0004,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
DT0005,0,1,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT0396,1,1,0,1,0,1,1,0,1,0,...,1,1,0,0,0,1,1,0,0,0
DT0397,1,0,1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
DT0398,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
DT0399,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### GenerateRandomTests function
We also include a `GenerateRandomTests()` function that is the same as the one in the `generateTest.ipynb` notebook. You can then call the `GenerateRandomTests()` function as many times as you want to re-generate a test. It will generate different results every time (unless you pass in an integer seed value).

Add items and candidates to the data files to generate larger tests.

When you are happy with the results you can write out to a results CSV file.

In [7]:
def GenerateRandomTests(seed: int = None):
    synthetic_test = generateTest(items, candidates, seed)
    testDf = convertTestToDataframe(synthetic_test, items, candidates)
    return testDf

In [8]:
df = GenerateRandomTests(89)

(df)

,A1L_7616_01#6789,A1L_20679_02#6790,A1L_5480_03#6791,A2L_5483_04#6792,A2L_24442_05#6793,A2L_7620_06#6794,A2L_7627_07#6795,B1L_20849_08#6796,B1L_4287_09#6797,B1L_21135_10#6798,...,B1VG_3919_SA06,B2VG_5843_SA07,C1VG_4378_SA08,C2VG_8173_SA09,C2VG_18944_SA10,B2R_4464_WA_01,B2R_4161_WA_02,C1R_4135_WA_03,C1R_4421_WA_04,C1R_4136_WA_05
DT0001,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
DT0002,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
DT0003,1,1,1,1,1,0,0,1,1,1,...,1,1,0,1,1,0,1,0,0,0
DT0004,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DT0005,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT0396,1,1,0,1,1,1,1,1,1,1,...,1,1,1,0,0,1,1,0,0,0
DT0397,0,1,0,0,1,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
DT0398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DT0399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df.to_csv('data/results.csv', index=True)

### Solving using Standard Estimation
You can use either maximum marginal likelihood (MML) or joint maximum likelihood (JML) estimation methods with the Girth library. Here we use MML estimation for the 1PL model (`onepl_mml`) to separately estimate the item parameters (`a` is the discrimination parameter and `bs` is an array of item difficulties) using univariate optimization methods. 

In [10]:
estimates = onepl_mml(synthetic_test)

a = estimates['Discrimination']
bs = estimates['Difficulty']

In [11]:
(a)

2.3008121693009786

In [12]:
(bs)

array([ 0.23656492,  0.18090252,  0.21265742,  0.35762581,  0.26858168,
        0.40698576,  0.36583801,  0.45699424,  0.56809858,  0.55072805,
        0.58558573,  0.55072805,  0.78704391,  1.16199057,  0.99022939,
        0.99022939,  0.39046171,  0.34131331,  0.43190173,  0.42357769,
        0.40698576,  0.26858168,  0.46540012,  0.43190173,  0.47382769,
        0.38222637,  0.39046171,  0.38222637,  0.39046171,  0.38222637,
        0.39046171,  0.46540012,  0.33318059,  0.35762581,  0.34946214,
        0.39046171,  0.46540012,  0.38222637,  0.40698576,  0.44024556,
        0.48229017,  0.35762581,  0.32505867,  0.50776949,  0.38222637,
        0.39046171,  0.36583801,  0.308859  ,  0.41527293,  0.44024556,
       -0.06272453,  0.20470642,  0.23656492,  0.36583801,  0.308859  ,
        0.41527293,  0.50776949,  0.45699424,  0.51631644,  0.55939667,
        0.40698576,  0.48229017,  0.40698576,  0.41527293,  0.32505867,
        0.35762581,  0.39046171,  0.47382769,  0.40698576,  0.34